1. SQL Challenge
*Scenario:* You have a sales database with two tables: Orders (OrderID, CustomerID, OrderDate, Amount) and Customers (CustomerID, Name, City). Write SQL queries to:
- Find the total sales amount for each customer.
- List customers who have not placed any orders.
- Identify the city with the highest number of orders.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine, text
engine = create_engine('xxxxxcredentialsxxxxx')
conn = engine.connect()

In [2]:
customers = pd.read_sql_query(text('SELECT * FROM db_customers_kaggle'), conn)
customers.head()

,customerid,name,city,country
0,103,Atelier graphique,Nantes,France
1,112,Signal Gift Stores,Las Vegas,USA
2,114,"Australian Collectors, Co.",Melbourne,Australia
3,119,La Rochelle Gifts,Nantes,France
4,121,Baane Mini Imports,Stavern,Norway


In [3]:
orders = pd.read_sql_query(text('SELECT * FROM db_orders_kaggle'), conn)
orders.head()

,orderid,customerid,orderdate,amount
0,10100,363,2003-01-06,10224
1,10101,128,2003-01-09,10549
2,10102,181,2003-01-10,5495
3,10103,121,2003-01-29,50219
4,10104,141,2003-01-31,40206


In [4]:
total_sales_each_customers = pd.read_sql_query(text('select db_customers_kaggle.name, \
                                sum(db_orders_kaggle.amount) total_Sales \
                                from db_customers_kaggle \
                                inner join db_orders_kaggle on db_customers_kaggle.customerid = db_orders_kaggle.customerid\
                                group by 1\
                                order by 2 desc'), conn)
total_sales_each_customers


,name,total_sales
0,Euro+ Shopping Channel,820690
1,Mini Gifts Distributors Ltd.,591829
2,"Australian Collectors, Co.",180586
3,Muscle Machine Inc,177914
4,La Rochelle Gifts,158573
...,...,...
93,Royale Belge,29218
94,Frau da Collezione,25358
95,Atelier graphique,22314
96,Auto-Moto Classics Inc.,21555


In [5]:
customers_not_placed_any_orders = pd.read_sql_query(text('select db_customers_kaggle.name from db_customers_kaggle\
                                     left join db_orders_kaggle on db_customers_kaggle.customerid = db_orders_kaggle.customerid\
                                     where db_orders_kaggle.customerid is null\
                                     order by 1'), conn)
customers_not_placed_any_orders

,name
0,ANG Resellers
1,American Souvenirs Inc
2,"Anton Designs, Ltd."
3,"Asian Shopping Network, Co"
4,"Asian Treasures, Inc."
5,BG&E Collectables
6,"Cramer Spezialit�ten, Ltd"
7,Der Hund Imports
8,"Feuer Online Stores, Inc"
9,"Franken Gifts, Co"


In [6]:
city_highest_number_of_order = pd.read_sql_query(text('select db_customers_kaggle.city,\
                                     count (distinct db_orders_kaggle.orderid) number_of_orders\
                                     from db_customers_kaggle\
                                     inner join db_orders_kaggle on db_customers_kaggle.customerid = db_orders_kaggle.customerid\
                                     group by 1\
                                     order by 2 desc\
                                     limit 5 '), conn)
city_highest_number_of_order

,city,number_of_orders
0,Madrid,31
1,San Rafael,17
2,NYC,16
3,Auckland,9
4,Paris,9


2. Data Modeling Exercise
*Scenario:* Design a data model for an e-commerce platform that includes entities for users, products, orders, and reviews. Ensure that your model supports querying the top-rated products in each category and tracks the order history of users.


In [7]:
df = pd.read_excel('Flat.table.xlsx')

In [8]:
df.dropna(inplace=True)
df.head()

,id.product,product.name,order.id,order.date,revenue,reviews.date,reviews.id,reviews.rating,users.city,users.province,users.id,users
0,P1,Electronic accessories,1004,2017-01-10,10224,2017-01-13,R-5,5,Bolaang Mongondow Regency,"North Sulawesi, Indonesia",C312,Adapter
1,P2,Fashion accessories,1004,2017-01-10,10549,2017-01-13,R-5,5,Sigi Regency,"Central Sulawesi, Indonesia",C19650,truman
2,P3,Food and beverages,1004,2017-01-10,5495,2017-01-13,R-5,5,South Konawe Regency,"Southeast Sulawesi, Indonesia",C4529,DaveZ
3,P4,Health and beauty,1004,2017-01-10,50219,2017-01-13,R-4,4,Simalungun Regency,"North Sumatra, Indonesia",C17182,Shacks
4,P5,Home and lifestyle,1004,2017-01-09,40206,2017-01-12,R-5,5,Manado City,"North Sulawesi, Indonesia",C6039,explore42


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13148 entries, 0 to 13147
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id.product      13148 non-null  object        
 1   product.name    13148 non-null  object        
 2   order.id        13148 non-null  int64         
 3   order.date      13148 non-null  datetime64[ns]
 4   revenue         13148 non-null  int64         
 5   reviews.date    13148 non-null  datetime64[ns]
 6   reviews.id      13148 non-null  object        
 7   reviews.rating  13148 non-null  int64         
 8   users.city      13148 non-null  object        
 9   users.province  13148 non-null  object        
 10  users.id        13148 non-null  object        
 11  users           13148 non-null  object        
dtypes: datetime64[ns](2), int64(3), object(7)
memory usage: 1.2+ MB


1. Identifya Fact table (orders)
2. Determine Dimension table (users,products,and reviews)

In [10]:
orders_fact = df[["order.id","order.date","id.product","revenue","reviews.id","reviews.date","users.id"]]
orders_fact

,order.id,order.date,id.product,revenue,reviews.id,reviews.date,users.id
0,1004,2017-01-10,P1,10224,R-5,2017-01-13,C312
1,1004,2017-01-10,P2,10549,R-5,2017-01-13,C19650
2,1004,2017-01-10,P3,5495,R-5,2017-01-13,C4529
3,1004,2017-01-10,P4,50219,R-4,2017-01-13,C17182
4,1004,2017-01-09,P5,40206,R-5,2017-01-12,C6039
...,...,...,...,...,...,...,...
13143,1024,2017-02-11,P4,51619,R-5,2017-02-14,C8963
13144,1024,2017-02-11,P5,20314,R-5,2017-02-14,C617
13145,1024,2017-02-11,P6,35035,R-5,2017-02-14,C2680
13146,1024,2017-02-11,P1,6420,R-5,2017-02-14,C16460


In [11]:
products_dim = df[["id.product","product.name"]].drop_duplicates().sort_values(by=['id.product'], ascending=True)
products_dim

,id.product,product.name
0,P1,Electronic accessories
1,P2,Fashion accessories
2,P3,Food and beverages
3,P4,Health and beauty
4,P5,Home and lifestyle
5,P6,Sports and travel


In [12]:
users_dim = df[["users.id","users","users.city","users.province"]].drop_duplicates().sort_values(by=['users.id'], ascending=True).reset_index(drop=True)
users_dim

,users.id,users,users.city,users.province
0,C1,April,Bau-Bau City,"Southeast Sulawesi, Indonesia"
1,C10000,KDS4,Muaro Jambi Regency,"Jambi, Indonesia"
2,C10001,kealmoon,Mukomuko Regency,"Bengkulu, Indonesia"
3,C10004,Kedar,West Muna Regency,"Southeast Sulawesi, Indonesia"
4,C10005,Kedmsd,Murung Raya Regency,"Central Kalimantan, Indonesia"
...,...,...,...,...
10342,C9991,Kdawg,North Minahasa Regency,"North Sulawesi, Indonesia"
10343,C9993,Kdee,Mojokerto Regency,"East Java, Indonesia"
10344,C9994,Kden,Morowali Regency,"Central Sulawesi, Indonesia"
10345,C9996,kdla,North Morowali Regency,"Central Sulawesi, Indonesia"


In [13]:
reviews_dim = df[["reviews.id","reviews.rating"]].drop_duplicates().sort_values(by=['reviews.id'], ascending=True).reset_index(drop=True)
reviews_dim

,reviews.id,reviews.rating
0,R-1,1
1,R-2,2
2,R-3,3
3,R-4,4
4,R-5,5


Data modeling

In [14]:
from IPython.display import Image  
Image(url="data-modeling.png", width=500, height=250) 

3. ETL Process Task
*Scenario:* You are provided with a CSV file containing sales data (SaleID, SaleDate, ProductID, QuantitySold, UnitPrice) and a JSON file listing product details (ProductID, Name, Category). Write a script to extract data from these files, calculate the total sales amount for each product, and load the results into a new CSV file.


Sales data (csv file)

In [15]:
def total_sales_each_product():
    import json
    input1 = 'Sales.data.csv'
    input2 = 'Product.category.json'
    sales_data = pd.read_csv(input1,delimiter=';')
    sales_data["sales.date"] = pd.to_datetime(sales_data["sales.date"])
    sales_data["total_amount"] = sales_data["quantity.sold"]*sales_data["unit.prices"]
    with open('Product.category.json') as f:
        d = json.load(f)
        product_category = pd.read_json(d)
    total_sales_groupby_product = sales_data.groupby("product.id", as_index=False).sum()
    total_sales_each_product = pd.merge(product_category,total_sales_groupby_product, on='product.id', how='left')[["name","total_amount"]].sort_values(by=['total_amount'], ascending=False)
    total_sales_each_product = total_sales_each_product.reset_index(drop=True)
    
    return total_sales_each_product.to_csv("total_sales_each_product.csv"),total_sales_each_product
    
    
total_sales_each_product()

(None,
                                                name  total_amount
 0                    Sharp AL-1530CS Digital Copier       1633893
 1                                             2160i        956265
 2         Hoover WindTunnelâ„¢ Plus Canister Vacuum        909846
 3                Canon PC1060 Personal Laser Copier        902781
 4                     Canon Image Class D660 Copier        883728
 ..                                              ...           ...
 440                                        Avery 49            75
 441                                       Avery 498            75
 442     10- 4 1/8" x 9 1/2" Security-Tint Envelopes            62
 443  Personal Creationsâ„¢ Ink Jet Cards and Labels            58
 444                      Heavy-Duty E-Z-DÂ® Binders            50
 
 [445 rows x 2 columns])

4. Scripting and Automation
*Scenario:* Write a Python script that takes a URL of a webpage as input, scrapes the webpage for all hyperlinks, and outputs a list of unique URLs found on the page.


In [16]:
import pandas as pd
import requests
from bs4 import BeautifulSoup


def get_links(url):
    reqs = requests.get(url)
    soup = BeautifulSoup(reqs.text, 'html.parser')

    urls = []
    for link in soup.find_all('a'):
        print(link.get('href'))

get_links('https://www.cnnindonesia.com/nasional')

https://connect.detik.com/oauth/authorize?clientId=10027&redirectUrl=https%3A%2F%2Fwww.cnnindonesia.com%2Fauthorize%3Fu%3Dhttps%3A%2F%2Fwww.cnnindonesia.com%2Fnasional&backURL=https%3A%2F%2Fwww.cnnindonesia.com%2Fnasional
https://www.cnnindonesia.com/
https://www.cnnindonesia.com/
https://www.cnnindonesia.com/nasional
https://www.cnnindonesia.com/nasional/politik
https://www.cnnindonesia.com/nasional/hukum-kriminal
https://www.cnnindonesia.com/nasional/peristiwa
https://www.cnnindonesia.com/pemilu2024
https://www.cnnindonesia.com/nasional/info-politik
https://www.cnnindonesia.com/nasional/20240320075141-20-1076456/makam-sunan-kalijaga-di-demak-kebanjiran-warga-tetap-datang-ziarah
https://www.cnnindonesia.com/nasional/20240319200953-617-1076339/hasil-akhir-penghitungan-suara-pilpres-2024-di-pulau-sulawesi
https://www.cnnindonesia.com/nasional/20240320064858-617-1076426/prabowo-gibran-sapu-kemenangan-pilpres-2024-di-34-provinsi
https://www.cnnindonesia.com/internasional
https://www.cnnin

5. Performance Optimization
*Scenario:* You are provided with a database schema and a query that is running slow. The query involves joining three tables and filtering on a non-indexed column. Propose optimizations to improve the query's performance without significantly altering its output.


 Let find username, product id, total amount, and rating for each product

1. Without indexing execution time = 2.76 ms

In [17]:
from IPython.display import Image  
Image(url="not-indexes-column.png", width=500, height=250) 

2. Using indexing execution time = 0.25 ms (faster than without indexing)

In [18]:
from IPython.display import Image  
Image(url="indexes-column.png", width=500, height=250) 

6. Data Pipeline Design
*Scenario:* Design a data pipeline for processing web server logs to identify suspicious activities. The pipeline should collect logs, filter for specific patterns indicating potential security threats, and alert the security team.


On-progress

7. Dashboard Design
*Scenario:* Design a dashboard on POWER BI. You can free to showcase me any design which you think is good and relevant for any type of scenario.


Link dashboard = https://app.powerbi.com/view?r=eyJrIjoiODRiNWNmY2MtZTczOS00ZjBmLWJjNjQtYjBlYmJmZjQyMmVlIiwidCI6IjhmMDg4Y2I0LWUyNGMtNDAzMy04NDEyLTRiNjIzNjhiNzNkOSIsImMiOjEwfQ%3D%3D